# **Start**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Uninstall existing scikit-learn to avoid conflicts
!pip uninstall -y scikit-learn
# Install specific versions of libraries to avoid conflicts
!pip install scikit-learn==1.5.2
!pip install bayesian-optimization==3.2.0
!pip install optuna==4.6.0
!pip install gpboost==1.6.1
!pip install shap==0.50.0
!pip install ngboost==0.5.8
!pip install dask[dataframe]==2025.12.0
!pip install torch==2.9.0+cpu
!pip install seaborn==0.13.2
!pip install lightgbm==4.6.0
!pip install xgboost==3.1.2
!pip install lime==0.2.0.1
!pip install interpret==0.7.4
!pip install optunahub==0.4.0
!pip install cmaes==0.12.0
!pip install plotly==5.24.1
!pip install kaleido==1.2.0
!pip install openpyxl==3.1.5
!pip install properscoring==0.1
!pip install XlsxWriter==3.2.9
!pip install cython==3.0.12
!pip install pgbm==2.2.0
!pip install cp==2020.12.3
!pip install mapie==0.6.0
!pip install skorch==1.3.1
!pip install puncc==0.8.0
# Reinstall scikit-learn to the version required by ngboost
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.6.1
# Reinstall numpy first
!pip install numpy==1.26.4  # Use the version compatible with catboost
# Reinstall catboost
!pip install catboost==1.2.8

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 109.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
hdbscan 0.8.41 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.0 MB/s eta 0:00:00


In [ ]:
# Restart the runtime to apply changes
import os
os._exit(00)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ngboost
import gpboost
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from gpboost import GPBoostRegressor
from ngboost import NGBRegressor
import optuna
import optunahub
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from interpret import show
from interpret.blackbox import LimeTabular, ShapKernel
from optuna.samplers import RandomSampler
import random
import time
from ngboost.distns import Normal
from ngboost.scores import LogScore
from scipy.stats import norm
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
import plotly.express as px
from io import BytesIO
from openpyxl import Workbook, load_workbook
import os
from openpyxl.drawing.image import Image as openpyxlImage
import warnings
import xlsxwriter
from openpyxl.drawing.image import Image
from pgbm.sklearn import HistGradientBoostingRegressor
import torch
from pgbm.torch import PGBM
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import pickle
import json

In [3]:
train_data_path = "./drive/MyDrive/suspended sediment/SS_uncertainity/train.csv"
test_data_path = "./drive/MyDrive/suspended sediment/SS_uncertainity/test.csv"
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
print("Training data loaded successfully.")
print("Test data loaded successfully.")

Training data loaded successfully.
Test data loaded successfully.


In [4]:
print("\nShape of training data:", train_data.shape)
print("First 5 rows of training data:\n", train_data.head(5))
print("\nShape of test data:", test_data.shape)
print("First 5 rows of test data:\n", test_data.head(5))


Shape of training data: (1095, 4)
First 5 rows of training data:
       Qt   Qt-1    St-1      St
0  12.40  11.80  5930.0  5630.0
1   7.93  12.40  5630.0  1960.0
2   5.66   7.93  1960.0   491.0
3   5.04   5.66   491.0   692.0
4   4.84   5.04   692.0   452.0

Shape of test data: (365, 4)
First 5 rows of test data:
      Qt  Qt-1  St-1    St
0  5.27  5.49  45.0  44.0
1  5.13  5.27  44.0  29.0
2  4.84  5.13  29.0  23.0
3  5.04  4.84  23.0  26.0
4  5.21  5.04  26.0  30.0


In [5]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]
x_test= X_test
print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (1095, 3)
Shape of y_train: (1095,)
Shape of X_test: (365, 3)
Shape of y_test: (365,)


In [6]:
# Apply z-score normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Print the first five rows of the normalized data
print("\nFirst five rows of normalized X_train:")
print(X_train[:5])

print("\nFirst five rows of normalized X_test:")
print(X_test[:5])


First five rows of normalized X_train:
[[-0.13088871 -0.15525695  1.01508124]
 [-0.31063021 -0.131129    0.95334642]
 [-0.40190834 -0.31088224  0.19812377]
 [-0.42683893 -0.40216633 -0.10417107]
 [-0.43488105 -0.42709855 -0.06280874]]

First five rows of normalized X_test:
[[-0.41759048 -0.40900259 -0.19595018]
 [-0.42321997 -0.4178495  -0.19615596]
 [-0.43488105 -0.42347936 -0.1992427 ]
 [-0.42683893 -0.4351412  -0.2004774 ]
 [-0.42000312 -0.42709855 -0.19986005]]


# **Functions**

In [7]:
# Define the model classes
model_classes = {
    'Random Forest': RandomForestRegressor,
    'Gradient Boosting': GradientBoostingRegressor,
    'XGBoost': XGBRegressor,
    'LightGBM': LGBMRegressor,
    'GPBoost': GPBoostRegressor,
    'CatBoost': CatBoostRegressor,
    'NGBoost': NGBRegressor
}

feature_names = [f'Q{i}' for i in range(1, 3)]


In [8]:
def plot_best_scores(best_scores_ran, excel_file_path):
    # Extract the best pruner for each model based on RMSE and correlation coefficient
    best_rmse_scores = {}
    best_corr_coef_scores = {}

    for (model_name, pruner_name), scores in best_scores_ran.items():
        # Initialize if not already present
        if model_name not in best_rmse_scores:
            best_rmse_scores[model_name] = (scores['test_rmse'], pruner_name)
        if model_name not in best_corr_coef_scores:
            best_corr_coef_scores[model_name] = (scores['test_corr_coef'], pruner_name)

        # Update if better scores are found
        if scores['test_rmse'] < best_rmse_scores[model_name][0]:
            best_rmse_scores[model_name] = (scores['test_rmse'], pruner_name)
        if scores['test_corr_coef'] > best_corr_coef_scores[model_name][0]:
            best_corr_coef_scores[model_name] = (scores['test_corr_coef'], pruner_name)

    # Prepare data for plotting
    model_names_rmse = [f"{model} ({pruner})" for model, (rmse, pruner) in best_rmse_scores.items()]
    rmse_values = [rmse for rmse, _ in best_rmse_scores.values()]

    model_names_corr = [f"{model} ({pruner})" for model, (corr, pruner) in best_corr_coef_scores.items()]
    corr_values = [corr for corr, _ in best_corr_coef_scores.values()]

    # Plot RMSE
    plt.figure(figsize=(12, 6))
    bars_rmse = plt.bar(model_names_rmse, rmse_values, color='skyblue')

    # Highlight the best model
    best_rmse_index = np.argmin(rmse_values)
    bars_rmse[best_rmse_index].set_color('orange')

    # Annotate the bars with the RMSE scores
    for i, bar in enumerate(bars_rmse):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                 f'{rmse_values[i]:.5f}', ha='center', va='bottom', color='black')

    # Add labels and title for the RMSE bar chart
    plt.xlabel('Model (Pruner)')
    plt.ylabel('Test RMSE')
    plt.title('Best Test RMSE for Each Model')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    rmse_image_path = 'rmse_plot.png'
    plt.savefig(rmse_image_path)
    plt.close()

    # Plot Correlation Coefficient
    plt.figure(figsize=(12, 6))
    bars_corr = plt.bar(model_names_corr, corr_values, color='lightgreen')

    # Highlight the best model
    best_corr_index = np.argmax(corr_values)
    bars_corr[best_corr_index].set_color('orange')

    # Annotate the bars with the correlation coefficient scores
    for i, bar in enumerate(bars_corr):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.05,
                 f'{corr_values[i]:.5f}', ha='center', va='bottom', color='black')

    # Add labels and title for the correlation coefficient bar chart
    plt.xlabel('Model (Pruner)')
    plt.ylabel('Correlation Coefficient')
    plt.title('Best Correlation Coefficient for Each Model')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    corr_image_path = 'corr_plot.png'
    plt.savefig(corr_image_path)
    plt.close()

    # Load the existing Excel file
    workbook = load_workbook(excel_file_path)

    # Create a new sheet for the plots
    sheet_name = 'Best Models Plots'
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
    else:
        sheet = workbook.create_sheet(title=sheet_name)

    # Insert images into the new Excel sheet
    img_rmse = Image(rmse_image_path)
    img_corr = Image(corr_image_path)

    # Insert images
    sheet.add_image(img_rmse, 'A1')
    sheet.add_image(img_corr, 'A20')  # Adjust the position as needed

    # Save the workbook
    workbook.save(excel_file_path)

    # Clean up the image files
    os.remove(rmse_image_path)
    os.remove(corr_image_path)

# Example usage
# plot_best_scores(best_scores_ran, 'path_to_your_excel_file.xlsx')

In [9]:
def generate_interpretml_explanations_summary_pruners(
    results_dict, X_train, y_train, X_test, feature_names, instance_indices=None, excel_file_path=None
):
    if instance_indices is None:
        instance_indices = range(len(X_test))
    elif isinstance(instance_indices, int):
        instance_indices = [instance_indices]

    valid_indices = [idx for idx in instance_indices if 0 <= idx < len(X_test)]
    if not valid_indices:
        print("No valid instance indices provided.")
        return

    if isinstance(X_test, pd.DataFrame):
        instances_to_explain = X_test.iloc[valid_indices]
    else:
        instances_to_explain = X_test[valid_indices]

    best_model_pruners = {}
    for model_key, model_info in results_dict.items():
        if isinstance(model_key, tuple):
            model_name, pruner_name = model_key
        else:
            model_name = model_key
            pruner_name = None

        best_score = model_info.get('best_score')
        if best_score is None:
            print(f"No 'best_score' found for {model_key}. Skipping this combination.")
            continue

        if model_name not in best_model_pruners:
            best_model_pruners[model_name] = {
                'pruner_name': pruner_name,
                'model_info': model_info,
                'best_score': best_score
            }
        else:
            current_best_score = best_model_pruners[model_name]['best_score']
            if best_score < current_best_score:
                best_model_pruners[model_name] = {
                    'pruner_name': pruner_name,
                    'model_info': model_info,
                    'best_score': best_score
                }

    for model_name, info in best_model_pruners.items():
        pruner_name = info['pruner_name']
        model_info = info['model_info']
        best_params = model_info['best_params']
        model_class = model_classes.get(model_name)

        if model_class is None:
            print(f"Model {model_name} is not supported or not available.")
            continue

        if model_name == 'CatBoost':
            best_params['verbose'] = 0

        model = model_class(**best_params)
        model.fit(X_train, y_train)

        def predict_fn(data):
            return model.predict(data)

        if isinstance(X_train, pd.DataFrame):
            data_for_explainer = X_train.values
        else:
            data_for_explainer = X_train

        if isinstance(instances_to_explain, pd.DataFrame):
            data_for_explanation = instances_to_explain.values
        else:
            data_for_explanation = instances_to_explain

        # Generate LIME explanations
        lime_explainer = LimeTabular(
            predict_fn,
            data=data_for_explainer,
            feature_names=feature_names,
            random_state=1,
            mode='regression'
        )
        lime_explanation = lime_explainer.explain_local(data_for_explanation)

        feature_importances_lime = {}
        num_instances = len(valid_indices)

        for idx in range(num_instances):
            explanation = lime_explanation.data(idx)
            for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                feature_importances_lime[feature_name] = feature_importances_lime.get(feature_name, 0) + abs(feature_score)

        feature_importances_lime = {k: v / num_instances for k, v in feature_importances_lime.items()}
        feature_importances_lime = {k: round(v, 3) for k, v in feature_importances_lime.items()}

        # Generate SHAP explanations using ShapKernel
        try:
            shap_explainer = ShapKernel(predict_fn, data_for_explainer, feature_names=feature_names)
            shap_explanation = shap_explainer.explain_local(data_for_explanation)

            feature_importances_shap = {}
            for idx in range(num_instances):
                explanation = shap_explanation.data(idx)
                for feature_name, feature_score in zip(explanation['names'], explanation['scores']):
                    feature_importances_shap[feature_name] = feature_importances_shap.get(feature_name, 0) + abs(feature_score)

            feature_importances_shap = {k: v / num_instances for k, v in feature_importances_shap.items()}
            feature_importances_shap = {k: round(v, 3) for k, v in feature_importances_shap.items()}
        except Exception as e:
            print(f"Could not compute SHAP values for model {model_name}: {e}")
            continue

        # Plot LIME and SHAP feature importances side by side
        fig, axes = plt.subplots(1, 2, figsize=(34, 36))

        # Plot LIME feature importances
        lime_importances_df = pd.DataFrame.from_dict(
            feature_importances_lime, orient='index', columns=['importance']
        )
        lime_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        lime_importances_df.plot(kind='bar', legend=False, color='skyblue', ax=axes[0])
        axes[0].set_title(f"LIME Feature Importances for {model_name}")
        axes[0].set_ylabel("Average Absolute Importance Score")
        axes[0].set_xlabel("Features")
        axes[0].tick_params(axis='x', rotation=45)

        for p in axes[0].patches:
            height = p.get_height()
            axes[0].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        # Plot SHAP feature importances
        shap_importances_df = pd.DataFrame.from_dict(
            feature_importances_shap, orient='index', columns=['importance']
        )
        shap_importances_df.sort_values(by='importance', ascending=False, inplace=True)
        shap_importances_df.plot(kind='bar', legend=False, color='orange', ax=axes[1])
        axes[1].set_title(f"SHAP Feature Importances for {model_name}")
        axes[1].set_ylabel("Average Absolute SHAP Value")
        axes[1].set_xlabel("Features")
        axes[1].tick_params(axis='x', rotation=45)

        for p in axes[1].patches:
            height = p.get_height()
            axes[1].annotate(f'{height:.3f}',
                             (p.get_x() + p.get_width() / 2., height),
                             ha='center', va='bottom', fontsize=8)

        plt.tight_layout()

        # Save plots as images
        image_path = f'feature_importances_{model_name}.png'
        fig.savefig(image_path)
        plt.close(fig)

        # Optionally insert images and scores into an Excel file
        if excel_file_path:
            workbook = load_workbook(excel_file_path)
            sheet_name = f'{model_name} Explanations'
            if sheet_name in workbook.sheetnames:
                sheet = workbook[sheet_name]
            else:
                sheet = workbook.create_sheet(title=sheet_name)

            # Insert images into the new Excel sheet
            img = Image(image_path)
            sheet.add_image(img, 'A1')

            # Create a new sheet for feature importance scores
            scores_sheet_name = f'{model_name} Scores'
            if scores_sheet_name in workbook.sheetnames:
                scores_sheet = workbook[scores_sheet_name]
            else:
                scores_sheet = workbook.create_sheet(title=scores_sheet_name)

            # Write LIME scores
            scores_sheet.append(['Feature', 'LIME Importance'])
            for feature, importance in feature_importances_lime.items():
                scores_sheet.append([feature, importance])

            # Write SHAP scores
            scores_sheet.append(['Feature', 'SHAP Importance'])
            for feature, importance in feature_importances_shap.items():
                scores_sheet.append([feature, importance])

            # Save the workbook
            workbook.save(excel_file_path)

            # Clean up the image file
            os.remove(image_path)

# **Autosampler by Optuna**

In [13]:
import joblib

def mseloss_objective(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    gradient = yhat - y
    hessian = torch.ones_like(yhat)
    return gradient, hessian


def rmseloss_metric(yhat, y, sample_weight=None):
    if not torch.is_tensor(yhat):
        yhat = torch.from_numpy(np.array(yhat)).float()
    if not torch.is_tensor(y):
        y = torch.from_numpy(np.array(y)).float()
    loss = torch.sqrt(torch.mean((yhat - y) ** 2))
    return loss


def hyperparameter_tuning_all(X_train, y_train, X_test, y_test, excel_path):

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # ================= NEW =================
    model_save_dir = "./drive/MyDrive/suspended sediment/SS_uncertainity/SS_ml/HyperParameter_Tuning/models"
    os.makedirs(model_save_dir, exist_ok=True)
    best_rmse_tracker = {}
    # =======================================

    models = {
        'Random Forest': (RandomForestRegressor, {
            'n_estimators': [100, 200, 300, 500, 700],
            'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
            'max_depth': [None, 10, 20, 30, 40],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.1, 0.2],
            'max_features': [1.0, 'sqrt', 'log2', 0.3, 0.5],
            'max_leaf_nodes': [None, 50, 100, 200],
            'min_impurity_decrease': [0.0, 0.01, 0.1, 0.2],
            'n_jobs': [-1],
            'random_state': [42],
            'verbose': [0],
            'warm_start': [False],
            'ccp_alpha': [0.0, 0.001, 0.01, 0.05, 0.1]
        }),
        'Gradient Boosting': (GradientBoostingRegressor, {
            'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
            'learning_rate': [0.01, 0.05, 0.1, 0.2],
            'n_estimators': [100, 200, 300, 500, 700],
            'subsample': [1.0, 0.9, 0.7, 0.5],
            'criterion': ['friedman_mse', 'squared_error'],
            'min_samples_split': [2, 5, 10, 0.01],
            'min_samples_leaf': [1, 3, 5, 0.01],
            'min_weight_fraction_leaf': [0.0, 0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7, 10],
            'min_impurity_decrease': [0.0, 0.01, 0.1],
            'init': [None],
            'random_state': [42],
            'max_features': [None, 'sqrt', 'log2', 0.5],
            'alpha': [0.9, 0.5, 0.1],
            'verbose': [0],
            'max_leaf_nodes': [None, 10, 30, 50],
            'warm_start': [False],
            'validation_fraction': [0.1],
            'n_iter_no_change': [None, 10, 20],
            'tol': [1e-4, 1e-3],
            'ccp_alpha': [0.0, 0.001, 0.01]
        }),
        'XGBoost': (XGBRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3, 5],
            'gamma': [0, 0.1, 0.5, 1],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
            'colsample_bytree': [0.5, 0.7, 0.9],
            'colsample_bylevel': [0.5, 0.7, 0.9],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0.1, 1, 5, 10],
            'objective': ['reg:squarederror'],
            'random_state': [42],
            'n_jobs': [-1]
        }),
        'LightGBM': (LGBMRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'num_leaves': [15, 31, 63],
            'max_depth': [3, 5, 7, -1],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.01, 0.1, 1],
            'reg_lambda': [0, 0.1, 1, 10],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'bagging_freq': [0, 1, 5],
            'objective': ['regression'],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'GPBoost': (GPBoostRegressor, {
            'n_estimators': [100, 200, 300, 400, 500],
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_depth': [3, 5, 7, -1],
            'num_leaves': [15, 31, 63],
            'min_child_samples': [1, 5, 10, 20],
            'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
            'reg_alpha': [0, 0.1, 0.5, 1.0],
            'reg_lambda': [0, 0.1, 0.5, 1.0],
            'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1],
            'random_state': [42],
            'n_jobs': [-1],
            'verbose': [-1]
        }),
        'CatBoost': (CatBoostRegressor, {
            'iterations': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'depth': [4, 6, 8, 10],
            'l2_leaf_reg': [1, 3, 5, 7, 9],
            'border_count': [32, 64, 128],
            'min_data_in_leaf': [1, 5, 10, 20],
            'rsm': [0.6, 0.8, 1.0],
            'bagging_temperature': [0, 1, 10],
            'random_seed': [42],
            'verbose': [0]
        }),
        'NGBoost': (NGBRegressor, {
            'n_estimators': [200, 500, 1000],
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'natural_gradient': [True, False],
            'minibatch_frac': [0.5, 0.7, 0.9, 1.0],
            'col_sample': [0.5, 0.7, 0.9, 1.0],
            'Dist': [Normal],
            'Score': [LogScore],
            'random_state': [42],
            'verbose': [0]
        }),
        'HistGradientBoosting': (HistGradientBoostingRegressor, {
            'learning_rate': [0.01, 0.05, 0.1, 0.15],
            'max_iter': [100, 200, 300, 400, 500],
            'max_depth': [3, 5, 7, None],
            'min_samples_leaf': [5, 10, 20],
            'max_leaf_nodes': [15, 31, 63, None],
            'l2_regularization': [0.0, 0.1, 0.5, 1.0],
            'max_bins': [64, 128, 255],
            'early_stopping': [True, False],
            'validation_fraction': [0.1, 0.2],
            'n_iter_no_change': [5, 10, 15],
            'loss': ['squared_error'],
            'random_state': [42],
            'verbose': [0]
        }),
        'PGBM': (PGBM, {})
    }

    pruners = [
        optuna.pruners.MedianPruner(),
        optuna.pruners.NopPruner(),
        optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=3),
        optuna.pruners.PercentilePruner(25.0),
        optuna.pruners.SuccessiveHalvingPruner(),
        optuna.pruners.HyperbandPruner(),
        optuna.pruners.ThresholdPruner(lower=0.1),
        optuna.pruners.WilcoxonPruner()
    ]

    best_scores = {}
    predictions_df = pd.DataFrame({'Actual': y_test})
    timing_records = []

    for model_name, (model_class, param_space) in models.items():

        rmse_trial_history = {p.__class__.__name__: [] for p in pruners}

        for pruner in pruners:
            pruner_name = pruner.__class__.__name__
            print(f"Running Optuna for {model_name} with {pruner_name}...")
            start_time = time.time()

            best_rmse_tracker[(model_name, pruner_name)] = np.inf

            sampler = optunahub.load_module("samplers/auto_sampler").AutoSampler()
            study = optuna.create_study(direction='minimize', sampler=sampler, pruner=pruner)

            if model_name == 'PGBM':

                def pgbm_objective(trial):
                    params = {
                            'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 300, 500]),
                            'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1, 0.15]),
                            'max_leaves': trial.suggest_int('max_leaves', 15, 63),
                            'min_split_gain': trial.suggest_categorical('min_split_gain', [0.0, 0.1, 0.5, 1.0]),
                            'reg_lambda': trial.suggest_categorical('reg_lambda', [0.1, 1.0, 5.0, 10.0]),
                            'feature_fraction': trial.suggest_categorical('feature_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'bagging_fraction': trial.suggest_categorical('bagging_fraction', [0.5, 0.7, 0.9, 1.0]),
                            'tree_correlation': trial.suggest_categorical('tree_correlation', [0.0, 0.1, 0.2, 0.3]),
                            'min_data_in_leaf': trial.suggest_categorical('min_data_in_leaf', [3, 5, 10, 20]),
                            'max_bin': trial.suggest_categorical('max_bin', [64, 128, 256]),
                            'distribution': trial.suggest_categorical('distribution', ['normal', 'studentt', 'laplace']),
                            'objective': 'mse',
                            'metric': 'rmse',
                            'random_state': 42,
                            'verbose': 0
                        }

                    model = PGBM()
                    model.train((X_train, y_train),
                                objective=mseloss_objective,
                                metric=rmseloss_metric,
                                params=params)

                    y_pred = model.predict(X_test)
                    mse = mean_squared_error(y_test, y_pred)
                    rmse = np.sqrt(mse)

                    rmse_trial_history[pruner_name].append(rmse)

                    # ===== SAVE BEST MODEL =====
                    if rmse < best_rmse_tracker[(model_name, pruner_name)]:
                        best_rmse_tracker[(model_name, pruner_name)] = rmse
                        save_path = os.path.join(
                            model_save_dir,
                            f"{model_name}_{pruner_name}_BEST.pkl"
                        )
                        joblib.dump(model, save_path)

                    return mse

                study.optimize(pgbm_objective, n_trials=50)

            else:

                def objective(trial):
                    params = {}
                    for key, values in param_space.items():
                        params[key] = trial.suggest_categorical(key, values)

                    model = model_class(**params)
                    model.fit(X_train, y_train)

                    y_pred = model.predict(X_test)
                    mse = mean_squared_error(y_test, y_pred)
                    rmse = np.sqrt(mse)

                    rmse_trial_history[pruner_name].append(rmse)

                    # ===== SAVE BEST MODEL =====
                    if rmse < best_rmse_tracker[(model_name, pruner_name)]:
                        best_rmse_tracker[(model_name, pruner_name)] = rmse
                        save_path = os.path.join(
                            model_save_dir,
                            f"{model_name}_{pruner_name}_BEST.pkl"
                        )
                        joblib.dump(model, save_path)

                    return mse

                study.optimize(objective, n_trials=50)

            elapsed_time = time.time() - start_time

            # Load frozen model (NO RETRAIN)
            best_model = joblib.load(
                os.path.join(model_save_dir, f"{model_name}_{pruner_name}_BEST.pkl")
            )

            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            corr_coef = np.corrcoef(y_test, y_pred)[0, 1]

            predictions_df[f'{model_name}_{pruner_name}_Predicted'] = y_pred

            best_scores[(model_name, pruner_name)] = {
                'best_score': mse,
                'best_params': study.best_params,
                'test_mse': mse,
                'test_rmse': rmse,
                'test_corr_coef': corr_coef,
                'pruner': pruner_name
            }

            timing_records.append({
                'Model': model_name,
                'Pruner': pruner_name,
                'Tuning_Time_Seconds': elapsed_time
            })

        # RMSE plots & Excel writing (UNCHANGED)
        rmse_df = pd.DataFrame(rmse_trial_history)
        rmse_df.insert(0, "Trial", np.arange(1, len(rmse_df) + 1))

        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            rmse_df.to_excel(writer, sheet_name=f"RMSE_Trials_{model_name}", index=False)
        # ================= SAVE RMSE PLOT =================
        plot_dir = os.path.dirname(excel_path)
        plot_path = os.path.join(plot_dir, f"RMSE_Trials_{model_name}.png")

        plt.figure(figsize=(10, 6))
        for pruner_name, values in rmse_trial_history.items():
            if len(values) > 0:   # <-- important safety check
                plt.plot(values, label=pruner_name, linewidth=2)

        plt.title(f"RMSE Variation Over Trials\n{model_name}")
        plt.xlabel("Trial Number")
        plt.ylabel("RMSE")
        plt.grid(True, alpha=0.3)
        plt.legend(loc="center left", bbox_to_anchor=(1.02, 0.5))
        plt.tight_layout()

        plt.savefig(plot_path, dpi=100, bbox_inches="tight")
        plt.close()

        # ================= INSERT PLOT INTO EXCEL =================
        wb = load_workbook(excel_path)
        ws = wb[f"RMSE_Trials_{model_name}"]

        img = Image(plot_path)
        img.anchor = "J2"
        ws.add_image(img)

        wb.save(excel_path)

    timing_df = pd.DataFrame(timing_records)

    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
        predictions_df.to_excel(writer, sheet_name='Predictions', index=False)
        timing_df.to_excel(writer, sheet_name='Tuning_Time', index=False)

    return best_scores


best_scores_autosampler = hyperparameter_tuning_all(X_train, y_train, X_test, y_test, "./drive/MyDrive/suspended sediment/SS_uncertainity/SS_ml/HyperParameter_Tuning/test.xlsx")


[I 2025-12-28 04:55:53,018] A new study created in memory with name: no-name-8c3212a0-3bb3-4fb9-8264-c79847839153


Running Optuna for Random Forest with MedianPruner...


[I 2025-12-28 04:55:54,591] Trial 0 finished with value: 685791.1443777838 and parameters: {'n_estimators': 500, 'criterion': 'poisson', 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.2, 'max_features': 'log2', 'max_leaf_nodes': 100, 'min_impurity_decrease': 0.01, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.001}. Best is trial 0 with value: 685791.1443777838.
[I 2025-12-28 04:55:56,272] Trial 1 finished with value: 1219772.484250765 and parameters: {'n_estimators': 500, 'criterion': 'poisson', 'max_depth': 10, 'min_samples_split': 0.01, 'min_samples_leaf': 0.01, 'min_weight_fraction_leaf': 0.1, 'max_features': 0.5, 'max_leaf_nodes': 100, 'min_impurity_decrease': 0.01, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.0}. Best is trial 0 with value: 685791.1443777838.
[I 2025-12-28 04:55:56,779] Trial 2 finished with value: 248129.687203146 and parameters: {'n_esti

Running Optuna for Random Forest with NopPruner...


[I 2025-12-28 04:58:42,574] Trial 0 finished with value: 982675.1209127838 and parameters: {'n_estimators': 300, 'criterion': 'friedman_mse', 'max_depth': 40, 'min_samples_split': 10, 'min_samples_leaf': 0.01, 'min_weight_fraction_leaf': 0.2, 'max_features': 1.0, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.01, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.01}. Best is trial 0 with value: 982675.1209127838.
[I 2025-12-28 04:58:44,085] Trial 1 finished with value: 966399.3588446764 and parameters: {'n_estimators': 500, 'criterion': 'poisson', 'max_depth': 10, 'min_samples_split': 0.01, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.2, 'max_features': 1.0, 'max_leaf_nodes': 100, 'min_impurity_decrease': 0.2, 'n_jobs': -1, 'random_state': 42, 'verbose': 0, 'warm_start': False, 'ccp_alpha': 0.01}. Best is trial 1 with value: 966399.3588446764.
[I 2025-12-28 04:58:45,481] Trial 2 finished with value: 135086.95785215645 and parameters: {'n_es

...

[I 2025-12-28 07:51:21,584] Trial 47 finished with value: 89384.65119096026 and parameters: {'n_estimators': 500, 'learning_rate': 0.01, 'max_leaves': 62, 'min_split_gain': 1.0, 'reg_lambda': 10.0, 'feature_fraction': 1.0, 'bagging_fraction': 0.7, 'tree_correlation': 0.1, 'min_data_in_leaf': 10, 'max_bin': 64, 'distribution': 'normal'}. Best is trial 47 with value: 89384.65119096026.


Training on CPU


[I 2025-12-28 07:51:36,362] Trial 48 finished with value: 89384.65119096026 and parameters: {'n_estimators': 500, 'learning_rate': 0.01, 'max_leaves': 59, 'min_split_gain': 1.0, 'reg_lambda': 10.0, 'feature_fraction': 1.0, 'bagging_fraction': 0.7, 'tree_correlation': 0.1, 'min_data_in_leaf': 10, 'max_bin': 64, 'distribution': 'normal'}. Best is trial 47 with value: 89384.65119096026.


Training on CPU


[I 2025-12-28 07:51:59,457] Trial 49 finished with value: 94991.02087665536 and parameters: {'n_estimators': 500, 'learning_rate': 0.01, 'max_leaves': 60, 'min_split_gain': 1.0, 'reg_lambda': 10.0, 'feature_fraction': 1.0, 'bagging_fraction': 0.7, 'tree_correlation': 0.1, 'min_data_in_leaf': 5, 'max_bin': 64, 'distribution': 'normal'}. Best is trial 47 with value: 89384.65119096026.


In [14]:
best_scores_autosampler

{('Random Forest', 'MedianPruner'): {'best_score': 124910.2663347441,
  'best_params': {'n_estimators': 700,
   'criterion': 'absolute_error',
   'max_depth': 10,
   'min_samples_split': 0.01,
   'min_samples_leaf': 3,
   'min_weight_fraction_leaf': 0.01,
   'max_features': 'log2',
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.01,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   'ccp_alpha': 0.1},
  'test_mse': 124910.2663347441,
  'test_rmse': 353.4264652438242,
  'test_corr_coef': 0.9581636337642089,
  'pruner': 'MedianPruner'},
 ('Random Forest', 'NopPruner'): {'best_score': 124842.65407450899,
  'best_params': {'n_estimators': 700,
   'criterion': 'absolute_error',
   'max_depth': None,
   'min_samples_split': 2,
   'min_samples_leaf': 3,
   'min_weight_fraction_leaf': 0.01,
   'max_features': 0.3,
   'max_leaf_nodes': 50,
   'min_impurity_decrease': 0.0,
   'n_jobs': -1,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False,
   

# **Best Model Analysis**

In [15]:
def get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, file_path):
    # Convert input data to NumPy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    # Mapping for model creation based on dictionary keys
    model_mapping = {
        'Random Forest': RandomForestRegressor,
        'Gradient Boosting': GradientBoostingRegressor,
        'XGBoost': XGBRegressor,
        'LightGBM': LGBMRegressor,
        'CatBoost': CatBoostRegressor,
        'GPBoost': GPBoostRegressor,
        'NGBoost': NGBRegressor,
        'HistGradientBoosting': HistGradientBoostingRegressor,
        'PGBM': PGBM  # PGBM is handled separately
    }

    # Dictionary to store the best model for each type
    best_models = {}

    # Iterate over the dictionary to find the best pruner for each model type
    for (model_name, pruner), params in best_scores_autosampler.items():
        current_score = params.get('test_mse', np.inf)
        if model_name not in best_models or current_score < best_models[model_name]['score']:
            best_models[model_name] = {
                'score': current_score,
                'params': params['best_params'],
                'pruner': pruner
            }

    # Prepare a DataFrame to store predictions
    df = pd.read_csv(file_path)

    # Iterate over the best models to train and predict
    for model_name, model_info in best_models.items():
        best_params = model_info['params']
        model_class = model_mapping.get(model_name)

        if model_class is None:
            print(f"Model {model_name} is not supported or not available.")
            continue

        # Handle specific parameters or settings for model if needed
        if model_name == 'CatBoost':
            best_params.pop('verbose', None)  # Remove 'verbose' for CatBoost

        # Create an instance of the best model with the best parameters
        if model_name == 'PGBM':
            model = model_class()
            model.train((X_train, y_train), objective=mseloss_objective, metric=rmseloss_metric, params=best_params)
            predictions = model.predict(X_test)
        else:
            model = model_class(**best_params)
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

        # Add predictions to the DataFrame
        df[f'{model_name} Predictions'] = predictions

        # Plot actual vs. predicted
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, predictions, alpha=0.6)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', color='red', lw=2)
        plt.xlabel("Actual")
        plt.ylabel("Predicted")
        plt.title(f"Actual vs. Predicted Values ({model_name})")
        plt.grid(True)
        plt.tight_layout()

        # Save the plot temporarily
        plot_path = f'temp_plot_{model_name}.png'
        plt.savefig(plot_path)
        plt.close()

    # Save predictions and plots to Excel
    with pd.ExcelWriter(file_path.replace('.csv', '_results.xlsx'), engine='xlsxwriter') as writer:
        # Write data to Excel
        df.to_excel(writer, sheet_name='Data', index=False)

        # Get the xlsxwriter objects
        workbook = writer.book

        # Insert each plot into a separate worksheet
        for model_name in best_models.keys():
            # Shorten the worksheet name to fit within Excel's 31 character limit
            short_model_name = ''.join([word[0] for word in model_name.split()])  # e.g., 'RF' for 'Random Forest'
            sheet_name = f'{short_model_name}_Plot'

            worksheet = workbook.add_worksheet(sheet_name)
            writer.sheets[sheet_name] = worksheet
            plot_path = f'temp_plot_{model_name}.png'
            worksheet.insert_image('A1', plot_path)

    # Clean up temporary plot files
    for model_name in best_models.keys():
        os.remove(f'temp_plot_{model_name}.png')

    return df, best_models

# Call the function
df, best_models = get_best_models_and_predict(best_scores_autosampler, X_train, y_train, X_test, y_test, "./drive/MyDrive/suspended sediment/SS_uncertainity/SS_ml/HyperParameter_Tuning/test.csv")

0:	learn: 4774.5604879	total: 22.2ms	remaining: 4.43s
1:	learn: 4694.0209493	total: 28.2ms	remaining: 2.79s
2:	learn: 4611.9270228	total: 31.7ms	remaining: 2.08s
3:	learn: 4535.3073007	total: 35.7ms	remaining: 1.75s
4:	learn: 4464.0064133	total: 37ms	remaining: 1.44s
5:	learn: 4391.4263384	total: 40.5ms	remaining: 1.31s
6:	learn: 4323.4154989	total: 44.3ms	remaining: 1.22s
7:	learn: 4255.8829308	total: 44.8ms	remaining: 1.08s
8:	learn: 4193.2257650	total: 45.5ms	remaining: 966ms
9:	learn: 4131.8818022	total: 47.5ms	remaining: 903ms
10:	learn: 4073.8046698	total: 51ms	remaining: 876ms
11:	learn: 4014.9106484	total: 52.6ms	remaining: 824ms
12:	learn: 3960.9601613	total: 55.9ms	remaining: 803ms
13:	learn: 3906.5853858	total: 58.9ms	remaining: 782ms
14:	learn: 3856.7636388	total: 59.6ms	remaining: 736ms
15:	learn: 3806.7997604	total: 60.3ms	remaining: 693ms
16:	learn: 3758.5385054	total: 66.2ms	remaining: 712ms
17:	learn: 3711.5923220	total: 66.5ms	remaining: 673ms
18:	learn: 3670.2868221	

In [16]:
plot_best_scores(best_scores_autosampler,"./drive/MyDrive/suspended sediment/SS_uncertainity/SS_ml/HyperParameter_Tuning/test_results.xlsx")

In [17]:
feature_names = ['Qt', 'Qt-1', 'st-1']

In [18]:
generate_interpretml_explanations_summary_pruners(best_scores_autosampler, X_train, y_train, x_test, feature_names,excel_file_path = "./drive/MyDrive/suspended sediment/SS_uncertainity/SS_ml/HyperParameter_Tuning/test_results.xlsx")

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

Model HistGradientBoosting is not supported or not available.
Model PGBM is not supported or not available.
